In [31]:
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from lightgbm import LGBMRegressor
from perpetual import PerpetualBooster
from scipy.special import expit
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_validate, train_test_split

In [32]:
pd.set_option("display.max_rows", 1000)

In [ ]:
!python --version

In [ ]:
from importlib.metadata import version

print(f"numpy: {version('numpy')}")
print(f"optuna: {version('optuna')}")
print(f"lightgbm: {version('lightgbm')}")
print(f"scikit-learn: {version('scikit-learn')}")
print(f"perpetual: {version('perpetual')}")

In [35]:
df = sns.load_dataset("diamonds")

In [ ]:
df.dtypes

In [37]:
X = df.drop(columns=["price", "y"])
y = df["price"]

In [ ]:
X.shape

In [ ]:
X.dtypes

In [ ]:
X.nunique()

In [ ]:
X.head()

In [ ]:
cols = ["cut", "color", "clarity"]
X[cols] = X[cols].astype("category")
X.head()

In [43]:
seed = 42
n_estimators = 100
n_trials = 1

In [44]:
scoring = "neg_root_mean_squared_error"
metric_function = root_mean_squared_error
metric_name = "rmse"
LGBMBooster = LGBMRegressor
objective = "SquaredLoss"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")

In [ ]:
X_train.head()

In [47]:
best_cv_results = None
cv_results = None


def save_best_cv_results(study, trial):
    global best_cv_results
    if study.best_trial.number == trial.number:
        best_cv_results = cv_results

In [48]:
def objective_function(trial):
    global cv_results
    params = {
        "seed": seed,
        "verbosity": -1,
        "n_estimators": n_estimators,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 1e-6, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-6, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 1.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "max_depth": trial.suggest_int("max_depth", 3, 33),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1024),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 100),
    }
    model = LGBMBooster(**params)
    cv_results = cross_validate(
        model,
        X_train,
        y_train,
        cv=5,
        scoring=scoring,
        return_train_score=True,
        return_estimator=True,
    )
    return -1 * np.mean(cv_results["test_score"])

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="minimize", sampler=sampler)

In [ ]:
%%time
study.optimize(objective_function, n_trials=n_trials, callbacks=[save_best_cv_results])

In [ ]:
print(f"Number of finished trials: {len(study.trials)}")
print("Best trial:")
print(f"  Number: {study.best_trial.number}")
print(f"  Value: {study.best_trial.value}")
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
print(f"CV train scores: {-1 * best_cv_results['train_score']}")
print(
    f"CV train scores average : {round(np.mean(-1 * best_cv_results['train_score']), 6)}"
)
print(f"CV test scores: {-1 * best_cv_results['test_score']}")
print(
    f"CV test scores average : {round(np.mean(-1 * best_cv_results['test_score']), 6)}"
)

In [53]:
models = best_cv_results["estimator"]

In [ ]:
for i, model in enumerate(models):
    y_pred = (
        model.predict_proba(X_train)
        if metric_name == "log_loss"
        else model.predict(X_train)
    )
    print(
        f"Model {i}, train {metric_name}: {round(metric_function(y_train, y_pred), 6)}"
    )

In [ ]:
for i, model in enumerate(models):
    y_pred = (
        model.predict_proba(X_test)
        if metric_name == "log_loss"
        else model.predict(X_test)
    )
    print(f"Model {i}, test {metric_name}: {round(metric_function(y_test, y_pred), 6)}")

In [ ]:
if metric_name == "log_loss":
    y_pred = np.mean([model.predict_proba(X_train) for model in models], axis=0)
else:
    y_pred = np.mean([model.predict(X_train) for model in models], axis=0)
print(f"Train {metric_name}: {round(metric_function(y_train, y_pred), 6)}")

In [ ]:
if metric_name == "log_loss":
    y_pred = np.mean([model.predict_proba(X_test) for model in models], axis=0)
else:
    y_pred = np.mean([model.predict(X_test) for model in models], axis=0)
print(f"Test {metric_name}: {round(metric_function(y_test, y_pred), 6)}")

In [ ]:
model = PerpetualBooster(objective=objective)
model.fit(X_train, y_train, budget=1.0)

In [ ]:
if metric_name == "log_loss":
    y_pred = expit(model.predict(X_test))
else:
    y_pred = np.round(expit(model.predict(X_test)))
print(f"Test {metric_name}: {round(metric_function(y_test, y_pred), 6)}")

In [33]:
df_trees = model.trees_to_dataframe()

In [ ]:
df_trees.head(10)

In [ ]:
models[4].booster_.trees_to_dataframe().head(100)